<a href="https://colab.research.google.com/github/p1atdev/KawaiiDiffusion/blob/main/Waifu_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waifu Diffusion
### 使い方

上から順に再生ボタン押していく。
ポップアップとかメッセージがでたらそれに従う。

プロンプトは「指示文」ブロックのとこ。自由に変えて色々試してね。

## セットアップ / Set up

In [ ]:
!bash git config --global credential.helper store

In [ ]:
!nvidia-smi

Fri Sep 16 14:29:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy datasets
!pip install "ipywidgets>=7,<8"

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "hakurei/waifu-diffusion"

pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)  

## セーフティーチェックを外す/ Remove the safety check

NSFW な写真を生成できるようになりますが、Googleアカウントを停止されても文句言わないでください。

これを解除する理由としては、単に全然 NSFW でない、ピンクやペールオレンジが多く含まれた画像でも引っかかってしまい、生成の効率が悪いためです。

心配ならここのブロックをコメントアウトするか削除してね。


In [ ]:
def dummy(images, **kwargs): return images, False 
pipe.safety_checker = dummy

In [ ]:
pipe = pipe.to("cuda")

ファイル保存用


In [ ]:
!pip install path

## Google Drive に接続 / Connect to Google Drive
接続認証のポップアップが出る

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## 出力パス / Output path
Google Drive での出力パス

`/Diffusion/Waifu` は自分で好きなように変えて良い

In [ ]:
output_path = "drive/MyDrive/Diffusion/Waifu"

## 画像保存 / Saving images
生成方針ごとにフォルダ生成
回数ごとにフォルダ生成

In [ ]:
from path import Path

trial_title = "Kawaii_girl"
trial_count = 0

# 保存パス / Save path
out_dir = Path(output_path)  / trial_title

In [ ]:
import os

# 保存用フォルダの作成
os.makedirs(out_dir)

## 指示文 / Prompts

In [ ]:
# テンプレート / Template
prompt_template = [
]

# 指令文 / Core
prompt_core = [
    "a girl",
]



## パラメータ / Parameters

In [ ]:
base_seed = 2294699762 # 乱数機で生成した適当な数字。Google検索で乱数と入れれば生成できます。(0 〜 9_999_999_999 で生成)
steps = 30 # 数字を上げると、何回も絵を微調整することになる。30前後がおすすめです。あげても必ずしも綺麗になるとは限らない。

# 生成する数
count = 10

# 画像サイズ
width = 512
height = 768 # 512, 768, 1024 など。 正方形以外の方が kawaii になりやすい

いざ生成！

### 生成機の準備
実行は一度だけで良い

In [ ]:
from torch import autocast
import pytz

# 実行コード
def run_prompt(seed, prompt_list, width, height, num_inference_steps=30):
    prompt = ", ".join(prompt_list)
    print("Generating", seed)
    # print("\n".join(prompt_list))
    generator = torch.Generator("cuda").manual_seed(seed)
    with autocast("cuda"):
        image = pipe(prompt, width=width, height=height, num_inference_steps=num_inference_steps, generator=generator)["sample"][0]
    return image

# プロンプト保存用
def save_prompt(out_dir, prompt):
    with open(out_dir / f"{trial_count}_prompt.txt", mode="w") as fp:
        fp.write(prompt)

## 通常生成
指定されたシードから順に1ずつ足していきながら指定された個数画像を生成する

In [ ]:
trial_count += 1

prompt_list = prompt_core[:] + prompt_template[:]

In [ ]:
# プロンプトを保存
save_prompt(out_dir, ", ".join(prompt_list))

# 繰り返し実行
for k in range(count):
    seed = base_seed + k

    print(f"Generate seed:{base_seed:d}")

    # 画像生成
    image = run_prompt(seed, prompt_list, width, height, steps)

    # 画像表示(邪魔ならコメントアウト)
    display(image)

    # 画像保存(保存しなければコメントアウト)
    image.save(out_dir / f"{trial_count}_{seed:d}.png")


## バリエーション
同じシード、プロンプト、異なるステップ数で生成する



In [ ]:
trial_count += 1
out_dir = Path(output_path)  / trial_title

# シードのリスト
static_seeds = [
  5161512896,
  5161512903,
  5161512906
]

# ステップのリスト(固定)
# step_list = [20, 30, 40, 50, 64] # 36とか入れるとなぜかエラー吐く
step_list = [100]

for seed in static_seeds:
  for setp in step_list:
      print(f"Variation seed:{seed:d} step:{setp:d}")
      image = run_prompt(seed, prompt_list, width, height, setp)
      # display(image)
      image.save(out_dir / f"{trial_count}_{seed:d}_{setp:d}.png")

## 比較検証コード
同じ条件で、異なるワードを付与して結果を比較する

In [ ]:
trial_count += 1
out_dir = Path(output_path)  / trial_title

# シードのリスト
static_seeds = [
    8946629288
]

# ステップのリスト(固定)
static_step = 30

# 変えるワードリスト
words = [
    "",
    "neko"
]

# 説明文
def save_description(seed, step):
  prompt_description = prompt_core[:]
  prompt_description.append("[条件]")
  prompt_description += prompt_template[:]
  description_text = f"prompt: {', '.join(prompt_description)}\nseed:{seed}\nstep:{step}"
  save_prompt(out_dir, description_text)

for seed in static_seeds:
  #プロンプトの説明文の保存
  save_description(seed, static_step)

  for word in words:
      print(f"Difference seed:{static_step:d} step:{static_step:d} word:{word}")
      prompt_copy = prompt_core[:]
      if word != "":
        prompt_copy.append(word)
      prompt_copy += prompt_template[:]

      print(prompt_copy)

      image = run_prompt(seed, prompt_copy, width, height, static_step)

      display(image)
      image.save(out_dir / f"{trial_count}_{seed:d}_{static_step:d}_{word}.png")

## ユーティリティ

文章をカンマで区切って出力してくれるやつ。
Lexicaのプロンプトをコピペしたい時用。

In [ ]:
raw_text = """
"""

result = raw_text.strip().split(",")

print("[")

for text in result:
  print(f'"{text.strip()}",')

print("]")